In [ ]:
import flask
from flask import make_response, jsonify
import json
from typing import Union, Any, Dict
from plaid import ApiException
from plaid.model.item_public_token_exchange_request import \
    ItemPublicTokenExchangeRequest
from plaid.model.item_public_token_exchange_response import \
    ItemPublicTokenExchangeResponse

from test_configuration import plaid_client, firestore_client

In [ ]:
def public_token_exchange(public_token: str) -> Dict[str, Any]:
    try:
        # Make Request to exchange `public_token` to `access_token`
        request = ItemPublicTokenExchangeRequest(public_token)
        response: ItemPublicTokenExchangeResponse = plaid_client.item_public_token_exchange(
            request
        )

        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def link_and_create_update_mode(request: flask.Request):
    # TODO: for release
    # Parsing data from request to get `uid` and `public_token`
    data_dict: dict = json.loads(request.data)
    public_token: Union[str, None] = data_dict.get('public_token')
    uid: Union[str, None] = data_dict.get('uid')

    if (public_token and uid) is None:
        return make_response(jsonify(error_code=404, error_message='Please pass the right data in request'), 404)
    
    # 1. Get access_token by using `item_public_token_exchange`
    token_exchange_response = public_token_exchange(public_token)
    # TODO: change the data passed from `institution_id` to `public_token`

    # access_token = token_exchange_response.get('access_token')
    return token_exchange_response
